<a href="https://colab.research.google.com/github/SrikanthArgp/colab_practices/blob/main/SentimentEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sentence_transformers import SentenceTransformer

In [ ]:
!pip install sentence-transformers

In [ ]:
twitter_file = '/content/drive/MyDrive/AppliedNLPMaterial-master/010_IntroToNLP/data/Tweets.csv'
df = pd.read_csv(twitter_file).dropna()
df

In [ ]:
cat_id = {'neutral': 1,
          'negative': 0,
          'positive': 2}

df['class'] = df['sentiment'].map(cat_id)

In [ ]:
BATCH_SIZE = 128
NUM_EPOCHS = 150
MAX_FEATURES = 10

In [ ]:
emb_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v1')

sentences = [ "Each sentence is converted"]
embeddings = emb_model.encode(sentences)
print(embeddings.squeeze().shape)

In [ ]:
!ls

In [ ]:
X = emb_model.encode(df['text'].values)

In [ ]:
with open("/content/drive/MyDrive/AppliedNLPMaterial-master/010_IntroToNLP/data/tweets_X.pkl", "wb") as output_file:
    pickle.dump(X, output_file)

In [ ]:
with open("/content/drive/MyDrive/AppliedNLPMaterial-master/010_IntroToNLP/data/tweets_X.pkl", "rb") as input_file:
    X = pickle.load(input_file)

In [ ]:
y = df['class'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=123)

In [ ]:
X_train[0].shape

In [ ]:
class SentimentData(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = torch.Tensor(X)
        self.y = torch.Tensor(y).type(torch.LongTensor)
        self.len = len(self.X)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [ ]:
train_ds = SentimentData(X= X_train, y = y_train)
test_ds = SentimentData(X_test, y_test)

In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=15000)

In [ ]:
class SentimentModel(nn.Module):
    def __init__(self, NUM_FEATURES, NUM_CLASSES, HIDDEN = 10):
        super().__init__()
        self.linear = nn.Linear(NUM_FEATURES, HIDDEN)
        self.linear2 = nn.Linear(HIDDEN, NUM_CLASSES)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.log_softmax(x)
        return x

In [ ]:
model = SentimentModel(NUM_FEATURES = X_train.shape[1], NUM_CLASSES = 3)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

In [ ]:
train_losses = []
for e in range(NUM_EPOCHS):
    curr_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred_log = model(X_batch)
        loss = criterion(y_pred_log, y_batch.long())

        curr_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_losses.append(curr_loss)
    print(f"Epoch {e}, Loss: {curr_loss}")

In [ ]:
sns.lineplot(x=list(range(len(train_losses))), y= train_losses)

In [ ]:
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_test_pred_log = model(X_batch)
        y_test_pred = torch.argmax(y_test_pred_log, dim = 1)

In [ ]:
y_test_pred_np = y_test_pred.squeeze().cpu().numpy()

In [ ]:
acc = accuracy_score(y_pred=y_test_pred_np, y_true = y_test)
f"The accuracy of the model is {np.round(acc, 3)*100}%."

In [ ]:
most_common_cnt = Counter(y_test).most_common()[0][1]
print(f"Naive Classifier: {np.round(most_common_cnt / len(y_test) * 100, 1)} %")